# 🌾 Rice Disease Detection - Model Training (Google Colab)

This notebook will train the CNN model on your rice disease dataset.

**Your dataset:** Already linked from Google Drive! ✅

**Steps:**
1. Click `Runtime → Change runtime type → GPU`
2. Run all cells (Ctrl+F9 or Runtime → Run all)
3. Wait 10-15 minutes for training
4. Download the trained model files
5. Place in your local `models/` folder

In [ ]:
# Install dependencies
!pip install -q tensorflow numpy pillow matplotlib seaborn scikit-learn gdown

In [ ]:
# Download and unzip dataset from Google Drive
import zipfile
import os
import gdown

# Your dataset file ID from Google Drive
file_id = '1YcJ_spCXpE9IfvhuLz31h2kzx5oF9Dky'
output_zip = '/content/rice_dataset.zip'

# Download from Google Drive
print("📥 Downloading dataset from Google Drive...")
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, output_zip, quiet=False)

# Extract dataset
print("\n📦 Extracting dataset...")
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/')

print("\n✅ Dataset ready!")
!ls -la /content/

## ⚠️ Important Note
After running the download cell above, check the output to see the exact folder name. If the folder is named differently (like "rice_dataset" instead of "Rice Leaf Disease Images"), the next cell will auto-detect it!

In [ ]:
# Training code
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Configuration
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Auto-detect dataset directory
import glob
possible_dirs = glob.glob('/content/*Rice*') + glob.glob('/content/*rice*')
if possible_dirs:
    DATA_DIR = possible_dirs[0]
    print(f"✅ Found dataset at: {DATA_DIR}")
else:
    DATA_DIR = "/content/Rice Leaf Disease Images"
    print(f"⚠️ Using default path: {DATA_DIR}")

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"🎮 GPU Available: {len(gpus) > 0}")
if gpus:
    print(f"   GPU: {gpus[0].name}")

In [ ]:
# Data Loading
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)
print(f"Classes: {class_names}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

In [ ]:
# Build Model
model = keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Block 4
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    # Dense layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

In [ ]:
# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

In [ ]:
# Save Model and Class Indices
model.save('rice_disease_model.h5')

class_indices = {v: k for k, v in train_generator.class_indices.items()}
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f, indent=4)

print("✓ Model saved")
print("✓ Class indices saved")

In [ ]:
# Evaluate
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"\nValidation Accuracy: {val_accuracy*100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

In [ ]:
# Confusion Matrix
val_generator.reset()
y_pred_proba = model.predict(val_generator)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = val_generator.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Training Curves
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Download files
from google.colab import files

print("Downloading trained model...")
files.download('rice_disease_model.h5')
files.download('class_indices.json')
files.download('confusion_matrix.png')
files.download('training_curves.png')

print("\n✓ All files downloaded!")
print("\nNext steps:")
print("1. Place 'rice_disease_model.h5' in your local 'models/' folder")
print("2. Place 'class_indices.json' in your local 'models/' folder")
print("3. Run: python app_simple.py")
print("4. Open: http://localhost:5000")

# 📊 Detailed Training Analysis & Metrics

In [ ]:
# Enhanced Training History Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create figure with multiple subplots
fig = plt.figure(figsize=(18, 12))

# 1. Accuracy Comparison
ax1 = plt.subplot(2, 3, 1)
plt.plot(history.history['accuracy'], marker='o', linewidth=2, label='Training Accuracy', color='#2ecc71')
plt.plot(history.history['val_accuracy'], marker='s', linewidth=2, label='Validation Accuracy', color='#e74c3c')
plt.title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.05])

# 2. Loss Comparison
ax2 = plt.subplot(2, 3, 2)
plt.plot(history.history['loss'], marker='o', linewidth=2, label='Training Loss', color='#3498db')
plt.plot(history.history['val_loss'], marker='s', linewidth=2, label='Validation Loss', color='#f39c12')
plt.title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True, alpha=0.3)

# 3. Accuracy Difference (Overfitting Detection)
ax3 = plt.subplot(2, 3, 3)
acc_diff = [train - val for train, val in zip(history.history['accuracy'], history.history['val_accuracy'])]
plt.plot(acc_diff, marker='d', linewidth=2, color='#9b59b6')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.title('Train-Val Accuracy Gap (Overfitting Check)', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy Difference', fontsize=12)
plt.grid(True, alpha=0.3)

# 4. Learning Rate Changes (if available)
ax4 = plt.subplot(2, 3, 4)
if 'lr' in history.history:
    plt.plot(history.history['lr'], marker='o', linewidth=2, color='#e67e22')
    plt.title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Learning Rate', fontsize=12)
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, 'Learning Rate\nNot Tracked', 
             ha='center', va='center', fontsize=14, transform=ax4.transAxes)
    plt.axis('off')

# 5. Metrics Bar Chart (Final Epoch)
ax5 = plt.subplot(2, 3, 5)
final_metrics = {
    'Train\nAccuracy': history.history['accuracy'][-1],
    'Val\nAccuracy': history.history['val_accuracy'][-1],
    'Train\nLoss': history.history['loss'][-1],
    'Val\nLoss': history.history['val_loss'][-1]
}
colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12']
bars = plt.bar(final_metrics.keys(), final_metrics.values(), color=colors, alpha=0.8, edgecolor='black')
plt.title('Final Epoch Metrics', fontsize=14, fontweight='bold')
plt.ylabel('Value', fontsize=12)
plt.xticks(rotation=0, fontsize=10)
# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 6. Best Epoch Highlight
ax6 = plt.subplot(2, 3, 6)
best_epoch = np.argmax(history.history['val_accuracy'])
best_val_acc = history.history['val_accuracy'][best_epoch]
best_val_loss = history.history['val_loss'][best_epoch]
best_train_acc = history.history['accuracy'][best_epoch]
best_train_loss = history.history['loss'][best_epoch]

info_text = f"""
BEST MODEL PERFORMANCE
━━━━━━━━━━━━━━━━━━━━━━━━━
Epoch: {best_epoch + 1}/{len(history.history['accuracy'])}

Validation Metrics:
  • Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)
  • Loss: {best_val_loss:.4f}

Training Metrics:
  • Accuracy: {best_train_acc:.4f} ({best_train_acc*100:.2f}%)
  • Loss: {best_train_loss:.4f}

Overfitting Gap: {(best_train_acc - best_val_acc)*100:.2f}%
"""
plt.text(0.1, 0.5, info_text, fontsize=11, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.axis('off')

plt.tight_layout()
plt.savefig('detailed_training_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Detailed training analysis graph saved!")

In [ ]:
# 📋 Table 1: Epoch-by-Epoch Training History
import pandas as pd
from IPython.display import display, HTML

# Create comprehensive training history DataFrame
epochs_data = {
    'Epoch': list(range(1, len(history.history['accuracy']) + 1)),
    'Training Accuracy': [f"{acc:.4f}" for acc in history.history['accuracy']],
    'Validation Accuracy': [f"{acc:.4f}" for acc in history.history['val_accuracy']],
    'Training Loss': [f"{loss:.4f}" for loss in history.history['loss']],
    'Validation Loss': [f"{loss:.4f}" for loss in history.history['val_loss']],
    'Acc Improvement': ['N/A'] + [f"{(history.history['val_accuracy'][i] - history.history['val_accuracy'][i-1])*100:+.2f}%" 
                                   for i in range(1, len(history.history['val_accuracy']))]
}

df_epochs = pd.DataFrame(epochs_data)

# Highlight best epoch
best_epoch_idx = np.argmax(history.history['val_accuracy'])

print("="*100)
print("📊 COMPLETE TRAINING HISTORY - EPOCH BY EPOCH")
print("="*100)
print(f"\n🏆 Best Epoch: {best_epoch_idx + 1} with Validation Accuracy: {history.history['val_accuracy'][best_epoch_idx]:.4f}\n")

# Style the dataframe
def highlight_best_epoch(row):
    if row['Epoch'] == best_epoch_idx + 1:
        return ['background-color: #90EE90; font-weight: bold'] * len(row)
    else:
        return [''] * len(row)

styled_df = df_epochs.style.apply(highlight_best_epoch, axis=1)\
    .set_properties(**{
        'text-align': 'center',
        'border': '1px solid black',
        'font-size': '11px'
    })\
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#4CAF50'), 
                                      ('color', 'white'), 
                                      ('font-weight', 'bold'),
                                      ('text-align', 'center'),
                                      ('border', '1px solid black')]}
    ])

display(styled_df)

# Save to CSV
df_epochs.to_csv('training_history.csv', index=False)
print("\n✅ Training history saved to 'training_history.csv'")

In [ ]:
# 📋 Table 2: Model Performance Summary
print("="*100)
print("📈 MODEL PERFORMANCE SUMMARY")
print("="*100)

# Training Configuration
config_data = {
    'Parameter': ['Total Epochs', 'Epochs Completed', 'Batch Size', 'Image Size', 
                  'Training Samples', 'Validation Samples', 'Classes', 'Learning Rate'],
    'Value': [
        EPOCHS,
        len(history.history['accuracy']),
        BATCH_SIZE,
        f"{IMG_SIZE}x{IMG_SIZE}",
        train_generator.samples,
        val_generator.samples,
        num_classes,
        LEARNING_RATE
    ]
}

df_config = pd.DataFrame(config_data)

print("\n🔧 TRAINING CONFIGURATION")
print("─"*50)
display(df_config.style.hide(axis='index').set_properties(**{
    'text-align': 'left',
    'border': '1px solid black',
    'font-size': '12px'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2196F3'), 
                                  ('color', 'white'), 
                                  ('font-weight', 'bold'),
                                  ('border', '1px solid black')]}
]))

# Performance Metrics
metrics_data = {
    'Metric': [
        'Best Validation Accuracy',
        'Best Validation Loss',
        'Final Validation Accuracy',
        'Final Validation Loss',
        'Best Training Accuracy',
        'Best Training Loss',
        'Final Training Accuracy',
        'Final Training Loss',
        'Best Epoch Number',
        'Overfitting Gap (Train-Val Acc)',
        'Model Converged',
        'Early Stopping Triggered'
    ],
    'Value': [
        f"{max(history.history['val_accuracy']):.4f} ({max(history.history['val_accuracy'])*100:.2f}%)",
        f"{min(history.history['val_loss']):.4f}",
        f"{history.history['val_accuracy'][-1]:.4f} ({history.history['val_accuracy'][-1]*100:.2f}%)",
        f"{history.history['val_loss'][-1]:.4f}",
        f"{max(history.history['accuracy']):.4f} ({max(history.history['accuracy'])*100:.2f}%)",
        f"{min(history.history['loss']):.4f}",
        f"{history.history['accuracy'][-1]:.4f} ({history.history['accuracy'][-1]*100:.2f}%)",
        f"{history.history['loss'][-1]:.4f}",
        np.argmax(history.history['val_accuracy']) + 1,
        f"{(max(history.history['accuracy']) - max(history.history['val_accuracy']))*100:.2f}%",
        "Yes" if len(history.history['accuracy']) < EPOCHS else "No (Reached max epochs)",
        "Yes" if len(history.history['accuracy']) < EPOCHS else "No"
    ]
}

df_metrics = pd.DataFrame(metrics_data)

print("\n📊 PERFORMANCE METRICS")
print("─"*50)
display(df_metrics.style.hide(axis='index').set_properties(**{
    'text-align': 'left',
    'border': '1px solid black',
    'font-size': '12px'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#FF9800'), 
                                  ('color', 'white'), 
                                  ('font-weight', 'bold'),
                                  ('border', '1px solid black')]}
]))

# Save summary
df_config.to_csv('model_configuration.csv', index=False)
df_metrics.to_csv('model_performance_metrics.csv', index=False)
print("\n✅ Configuration saved to 'model_configuration.csv'")
print("✅ Performance metrics saved to 'model_performance_metrics.csv'")

In [ ]:
# 📋 Table 3: Per-Class Performance Metrics
print("="*100)
print("🎯 PER-CLASS PERFORMANCE ANALYSIS")
print("="*100)

# Generate predictions for detailed metrics
val_generator.reset()
y_pred_proba = model.predict(val_generator, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = val_generator.classes

# Calculate per-class metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)

class_metrics = {
    'Disease Class': class_names,
    'Precision': [f"{p:.4f} ({p*100:.2f}%)" for p in precision],
    'Recall': [f"{r:.4f} ({r*100:.2f}%)" for r in recall],
    'F1-Score': [f"{f:.4f} ({f*100:.2f}%)" for f in f1],
    'Support (Samples)': support.astype(int),
    'Accuracy': [f"{accuracy_score(y_true[y_true==i], y_pred[y_true==i]):.4f}" 
                 if len(y_true[y_true==i]) > 0 else "N/A" for i in range(len(class_names))]
}

df_class_metrics = pd.DataFrame(class_metrics)

# Add average row
avg_row = pd.DataFrame({
    'Disease Class': ['OVERALL AVERAGE'],
    'Precision': [f"{precision.mean():.4f} ({precision.mean()*100:.2f}%)"],
    'Recall': [f"{recall.mean():.4f} ({recall.mean()*100:.2f}%)"],
    'F1-Score': [f"{f1.mean():.4f} ({f1.mean()*100:.2f}%)"],
    'Support (Samples)': [support.sum()],
    'Accuracy': [f"{accuracy_score(y_true, y_pred):.4f}"]
})

df_class_metrics = pd.concat([df_class_metrics, avg_row], ignore_index=True)

# Style the dataframe
styled_class = df_class_metrics.style.set_properties(**{
    'text-align': 'center',
    'border': '1px solid black',
    'font-size': '11px'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#9C27B0'), 
                                  ('color', 'white'), 
                                  ('font-weight', 'bold'),
                                  ('text-align', 'center'),
                                  ('border', '1px solid black')]}
]).apply(lambda x: ['background-color: #FFD700; font-weight: bold'] * len(x) 
         if x.name == len(df_class_metrics) - 1 else [''] * len(x), axis=1)

display(styled_class)

# Save to CSV
df_class_metrics.to_csv('per_class_performance.csv', index=False)
print("\n✅ Per-class performance saved to 'per_class_performance.csv'")

In [ ]:
# 📊 Advanced Accuracy Visualization with Statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Accuracy with Confidence Intervals
ax = axes[0, 0]
epochs_range = range(1, len(history.history['accuracy']) + 1)
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

ax.plot(epochs_range, train_acc, 'o-', linewidth=2.5, markersize=6, 
        label='Training Accuracy', color='#2ecc71', alpha=0.8)
ax.plot(epochs_range, val_acc, 's-', linewidth=2.5, markersize=6, 
        label='Validation Accuracy', color='#e74c3c', alpha=0.8)
ax.fill_between(epochs_range, train_acc, alpha=0.2, color='#2ecc71')
ax.fill_between(epochs_range, val_acc, alpha=0.2, color='#e74c3c')
ax.axhline(y=max(val_acc), color='red', linestyle='--', alpha=0.5, 
           label=f'Best Val Acc: {max(val_acc):.4f}')
ax.set_title('Model Accuracy Progress', fontsize=16, fontweight='bold')
ax.set_xlabel('Epoch', fontsize=13)
ax.set_ylabel('Accuracy', fontsize=13)
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_ylim([0, 1.05])

# Plot 2: Loss with Moving Average
ax = axes[0, 1]
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Calculate moving average (window=3)
def moving_average(data, window=3):
    return np.convolve(data, np.ones(window)/window, mode='valid')

if len(train_loss) >= 3:
    train_ma = moving_average(train_loss)
    val_ma = moving_average(val_loss)
    ma_epochs = range(2, len(train_loss) + 1)
    ax.plot(ma_epochs, train_ma, '--', linewidth=2, label='Train MA(3)', color='#3498db', alpha=0.6)
    ax.plot(ma_epochs, val_ma, '--', linewidth=2, label='Val MA(3)', color='#f39c12', alpha=0.6)

ax.plot(epochs_range, train_loss, 'o-', linewidth=2.5, markersize=6,
        label='Training Loss', color='#3498db', alpha=0.8)
ax.plot(epochs_range, val_loss, 's-', linewidth=2.5, markersize=6,
        label='Validation Loss', color='#f39c12', alpha=0.8)
ax.axhline(y=min(val_loss), color='orange', linestyle='--', alpha=0.5,
           label=f'Best Val Loss: {min(val_loss):.4f}')
ax.set_title('Model Loss with Moving Average', fontsize=16, fontweight='bold')
ax.set_xlabel('Epoch', fontsize=13)
ax.set_ylabel('Loss', fontsize=13)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

# Plot 3: Accuracy Improvement Rate
ax = axes[1, 0]
train_acc_change = [0] + [train_acc[i] - train_acc[i-1] for i in range(1, len(train_acc))]
val_acc_change = [0] + [val_acc[i] - val_acc[i-1] for i in range(1, len(val_acc))]

ax.bar([e - 0.2 for e in epochs_range], train_acc_change, width=0.4, 
       label='Training Δ Accuracy', color='#2ecc71', alpha=0.7)
ax.bar([e + 0.2 for e in epochs_range], val_acc_change, width=0.4,
       label='Validation Δ Accuracy', color='#e74c3c', alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_title('Epoch-to-Epoch Accuracy Change', fontsize=16, fontweight='bold')
ax.set_xlabel('Epoch', fontsize=13)
ax.set_ylabel('Change in Accuracy', fontsize=13)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Performance Comparison Box Plot
ax = axes[1, 1]
performance_data = [
    train_acc,
    val_acc,
    [1-l for l in train_loss],  # Convert loss to accuracy-like metric
    [1-l for l in val_loss]
]
box = ax.boxplot(performance_data, labels=['Train\nAccuracy', 'Val\nAccuracy', 
                                             'Train\n(1-Loss)', 'Val\n(1-Loss)'],
                 patch_artist=True, showmeans=True)

colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax.set_title('Training Metrics Distribution', fontsize=16, fontweight='bold')
ax.set_ylabel('Value', fontsize=13)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('advanced_accuracy_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Advanced accuracy analysis saved!")

In [ ]:
# 📥 Download All Analysis Files
from google.colab import files

print("="*80)
print("📥 DOWNLOADING ALL TRAINING ANALYSIS FILES")
print("="*80)

files_to_download = [
    'rice_disease_model.h5',
    'class_indices.json',
    'confusion_matrix.png',
    'training_curves.png',
    'detailed_training_analysis.png',
    'advanced_accuracy_analysis.png',
    'training_history.csv',
    'model_configuration.csv',
    'model_performance_metrics.csv',
    'per_class_performance.csv'
]

print("\n📦 Downloading files...\n")
for file in files_to_download:
    try:
        files.download(file)
        print(f"  ✅ {file}")
    except:
        print(f"  ⚠️ {file} - Not found (skipped)")

print("\n" + "="*80)
print("✨ TRAINING COMPLETE - ALL FILES DOWNLOADED!")
print("="*80)
print("\n📊 You now have:")
print("  • Model files (rice_disease_model.h5, class_indices.json)")
print("  • Visualizations (4 PNG files)")
print("  • Detailed tables (4 CSV files)")
print("\n🚀 Next Steps:")
print("  1. Place model files in your 'models/' folder")
print("  2. Review CSV files for detailed metrics")
print("  3. Run: python app_simple.py")
print("  4. Open: http://localhost:5000")
print("="*80)